<a href="https://www.kaggle.com/code/vtrackstar/best-defensive-nba-seasons?scriptVersionId=201939933" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-aba-baa-stats/Advanced.csv
/kaggle/input/nba-aba-baa-stats/Player Season Info.csv
/kaggle/input/nba-aba-baa-stats/Player Career Info.csv
/kaggle/input/nba-aba-baa-stats/Player Shooting.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per Game.csv
/kaggle/input/nba-aba-baa-stats/Player Totals.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Team Summaries.csv
/kaggle/input/nba-aba-baa-stats/Player Per Game.csv
/kaggle/input/nba-aba-baa-stats/All-Star Selections.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams (Voting).csv
/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams.csv
/kaggle/input/nba-aba-baa-stats/Team Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 36 Minutes.csv
/kaggle/input/nba-aba-baa-stats/Opponent Totals.csv
/kaggle/input/nba-aba-baa-stats/Team Abbrev.csv
/kaggle/input/nba-aba-b

In [2]:
playerpg = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Per Game.csv')
playerpbp = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv')
playeraw = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Award Shares.csv')
playersh = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Shooting.csv')
playertot = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Totals.csv')

print(playerpg.columns)
print(playerpbp.columns)
print(playeraw.columns)
print(playersh.columns)
print(playertot.columns)

Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'gs', 'mp_per_game', 'fg_per_game',
       'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game',
       'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent',
       'e_fg_percent', 'ft_per_game', 'fta_per_game', 'ft_percent',
       'orb_per_game', 'drb_per_game', 'trb_per_game', 'ast_per_game',
       'stl_per_game', 'blk_per_game', 'tov_per_game', 'pf_per_game',
       'pts_per_game'],
      dtype='object')
Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'mp', 'pg_percent', 'sg_percent',
       'sf_percent', 'pf_percent', 'c_percent',
       'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss',
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_gener

In [3]:
defpg = playertot[['seas_id', 'season', 'player', 'pos', 'tm', 'g', 'drb', 'stl', 'blk', 'pf']]
defpbp = playerpbp[['seas_id', 'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss', 'shooting_foul_committed', 'offensive_foul_drawn']]

defplr = defpg.merge(defpbp, on='seas_id', how='outer')
defplar = defplr.dropna()
defplayr = defplar[defplar['g'] >= 60]
defplayer = defplayr.copy()
defplayer.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn
16713,16714,2000,A.C. Green,PF,LAL,82,326.0,53.0,18.0,127,7.7,-0.8,53.0,18.0
16715,16716,2000,Aaron McKie,SG,PHI,82,199.0,108.0,18.0,194,0.2,-2.0,75.0,8.0
16716,16717,2000,Aaron Williams,PF,WAS,81,250.0,41.0,92.0,234,1.6,8.1,120.0,16.0
16717,16718,2000,Adam Keefe,C,UTA,62,91.0,17.0,13.0,90,-2.0,-7.9,29.0,5.0
16718,16719,2000,Adonal Foyle,C,GSW,76,250.0,26.0,136.0,218,-9.7,-0.9,116.0,18.0


In [4]:
unique_values = defplayer['pos'].unique()
print(unique_values)

['PF' 'SG' 'C' 'PG' 'SF' 'PF-SF' 'PF-C' 'PG-SG' 'C-PF' 'SF-SG' 'SG-SF'
 'SG-PG' 'SG-PF' 'SF-PF']


In [5]:
def categorize(pos):
    if pos in ['SG', 'PG', 'PG-SG', 'SG-PG']:
        return 'Guard'
    elif pos in ['SF', 'SF-PF', 'PF-SF', 'SF-SG', 'SG-SF', 'SF-C', 'SG-PG-SF', 'SG-PF']:
        return 'Wing'
    else:
        return 'Big'

defplayer['pos_group'] = defplayer['pos'].apply(categorize)

defplayer.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group
16713,16714,2000,A.C. Green,PF,LAL,82,326.0,53.0,18.0,127,7.7,-0.8,53.0,18.0,Big
16715,16716,2000,Aaron McKie,SG,PHI,82,199.0,108.0,18.0,194,0.2,-2.0,75.0,8.0,Guard
16716,16717,2000,Aaron Williams,PF,WAS,81,250.0,41.0,92.0,234,1.6,8.1,120.0,16.0,Big
16717,16718,2000,Adam Keefe,C,UTA,62,91.0,17.0,13.0,90,-2.0,-7.9,29.0,5.0,Big
16718,16719,2000,Adonal Foyle,C,GSW,76,250.0,26.0,136.0,218,-9.7,-0.9,116.0,18.0,Big


In [6]:
df1 = defplayer[defplayer['pos_group'] == 'Guard']
defguard = df1.copy()
defguard['score'] = (defguard['drb'] * 0.01) + (defguard['stl'] * 0.3) + (defguard['blk'] * 0.15) + (defguard['shooting_foul_committed'] * -0.25) + (defguard['offensive_foul_drawn'] * 0.15)
defguard.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
16715,16716,2000,Aaron McKie,SG,PHI,82,199.0,108.0,18.0,194,0.2,-2.0,75.0,8.0,Guard,19.54
16719,16720,2000,Adrian Griffin,SG,BOS,72,244.0,116.0,15.0,222,-1.1,-1.0,95.0,14.0,Guard,17.84
16723,16724,2000,Allan Houston,SG,NYK,82,233.0,65.0,14.0,219,2.8,4.1,85.0,29.0,Guard,7.03
16724,16725,2000,Allen Iverson,SG,PHI,70,196.0,144.0,5.0,162,1.0,-0.3,49.0,33.0,Guard,38.61
16728,16729,2000,Andre Miller,PG,CLE,82,195.0,84.0,17.0,194,-3.6,5.0,66.0,28.0,Guard,17.40


In [7]:
df2 = defplayer[defplayer['pos_group'] == 'Wing']
defwing = df2.copy()
defwing['score'] = (defwing['drb'] * 0.01) + (defwing['stl'] * 0.4) + (defwing['blk'] * 0.3) + (defwing['shooting_foul_committed'] * -0.35) + (defwing['offensive_foul_drawn'] * 0.1)
defwing.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
16738,16739,2000,Anthony Mason,SF,CHH,82,554.0,74.0,29.0,220,3.4,1.8,109.0,25.0,Wing,8.19
16764,16765,2000,Billy Owens,PF-SF,TOT,62,202.0,33.0,21.0,166,-2.5,-0.8,59.0,4.0,Wing,1.27
16772,16773,2000,Bonzi Wells,SF,POR,66,104.0,69.0,12.0,153,6.5,-1.9,47.0,7.0,Wing,16.49
16780,16781,2000,Bruce Bowen,SF,TOT,69,69.0,23.0,15.0,118,3.3,-0.7,29.0,13.0,Wing,5.54
16787,16788,2000,Bryon Russell,SF,UTA,82,328.0,128.0,23.0,255,9.0,16.0,99.0,50.0,Wing,31.73


In [8]:
df3 = defplayer[defplayer['pos_group'] == 'Big']
defbig = df3.copy()
defbig['score'] = (defbig['drb'] * 0.3) + (defbig['stl'] * 0.1) + (defbig['blk'] * 0.3) + (defbig['shooting_foul_committed'] * -0.1) + (defbig['offensive_foul_drawn'] * 0.05)
defbig.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
16713,16714,2000,A.C. Green,PF,LAL,82,326.0,53.0,18.0,127,7.7,-0.8,53.0,18.0,Big,104.10
16716,16717,2000,Aaron Williams,PF,WAS,81,250.0,41.0,92.0,234,1.6,8.1,120.0,16.0,Big,95.50
16717,16718,2000,Adam Keefe,C,UTA,62,91.0,17.0,13.0,90,-2.0,-7.9,29.0,5.0,Big,30.25
16718,16719,2000,Adonal Foyle,C,GSW,76,250.0,26.0,136.0,218,-9.7,-0.9,116.0,18.0,Big,107.70
16721,16722,2000,Alan Henderson,PF,ATL,82,306.0,81.0,54.0,233,-6.0,-3.9,116.0,15.0,Big,105.25


In [9]:
sortguard = defguard.sort_values(by='score', ascending=False)
sortguard.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
21088,21089,2008,Chris Paul,PG,NOH,80,259.0,217.0,4.0,185,8.1,9.1,36.0,16.0,Guard,61.69
16885,16886,2000,Eddie Jones,SG,CHH,72,262.0,192.0,49.0,176,6.4,11.7,57.0,54.0,Guard,61.42
21690,21691,2009,Chris Paul,PG,NOH,78,363.0,216.0,10.0,212,6.5,19.6,53.0,22.0,Guard,59.98
21759,21760,2009,Dwyane Wade,SG,MIA,79,309.0,173.0,106.0,178,3.2,14.2,80.0,14.0,Guard,52.99
22840,22841,2011,Chris Paul,PG,NOH,80,289.0,188.0,5.0,196,4.5,13.9,52.0,35.0,Guard,52.29


In [10]:
sortwing = defwing.sort_values(by='score', ascending=False)
sortwing.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
19925,19926,2006,Andrei Kirilenko,SF,UTA,69,391.0,102.0,220.0,162,1.0,12.3,67.0,24.0,Wing,89.66
21908,21909,2009,LeBron James,SF,CLE,81,507.0,137.0,93.0,139,15.0,21.2,55.0,13.0,Wing,69.82
26525,26526,2017,Giannis Antetokounmpo,SF,MIL,80,558.0,131.0,151.0,246,1.0,3.2,103.0,7.0,Wing,67.93
20913,20914,2007,Shawn Marion,SF,PHO,80,613.0,156.0,122.0,216,9.7,8.3,115.0,11.0,Wing,65.98
28110,28111,2019,Paul George,SF,OKC,77,523.0,170.0,34.0,214,7.8,16.5,61.0,30.0,Wing,65.08


In [11]:
sortbig = defbig.sort_values(by='score', ascending=False)
sortbig.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
16866,16867,2000,Dikembe Mutombo,C,ATL,82,853.0,27.0,269.0,248,-1.5,13.1,103.0,19.0,Big,329.95
21349,21350,2008,Marcus Camby,C,DEN,79,807.0,85.0,285.0,216,3.8,1.0,145.0,33.0,Big,323.25
21156,21157,2008,Dwight Howard,C,ORL,82,882.0,74.0,176.0,274,7.5,8.0,133.0,8.0,Big,311.90
22327,22328,2010,Dwight Howard,C,ORL,82,798.0,75.0,228.0,287,11.0,9.4,134.0,3.0,Big,302.05
25235,25236,2015,DeAndre Jordan,C,LAC,82,829.0,81.0,183.0,245,11.8,15.4,128.0,4.0,Big,299.10


In [12]:
best_guard = sortguard.drop_duplicates(subset='player', keep='first')
best_guard.head(10)

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
21088,21089,2008,Chris Paul,PG,NOH,80,259.0,217.0,4.0,185,8.1,9.1,36.0,16.0,Guard,61.69
16885,16886,2000,Eddie Jones,SG,CHH,72,262.0,192.0,49.0,176,6.4,11.7,57.0,54.0,Guard,61.42
21759,21760,2009,Dwyane Wade,SG,MIA,79,309.0,173.0,106.0,178,3.2,14.2,80.0,14.0,Guard,52.99
16842,16843,2000,Darrell Armstrong,PG,ORL,82,205.0,169.0,9.0,137,-0.3,-2.4,34.0,34.0,Guard,50.70
27553,27554,2018,Victor Oladipo,SG,IND,75,343.0,177.0,57.0,175,6.4,13.7,71.0,10.0,Guard,48.83
20992,20993,2008,Allen Iverson,SG,DEN,82,196.0,160.0,12.0,109,4.9,9.8,42.0,46.0,Guard,48.16
22600,22601,2010,Rajon Rondo,PG,BOS,81,260.0,189.0,11.0,198,4.7,2.4,62.0,12.0,Guard,47.25
16909,16910,2000,Gary Payton,PG,SEA,82,429.0,153.0,18.0,178,3.7,14.7,41.0,30.0,Guard,47.14
24982,24983,2014,Ricky Rubio,PG,MIN,82,281.0,191.0,11.0,218,6.6,12.6,82.0,36.0,Guard,46.66
26228,26229,2016,Russell Westbrook,PG,OKC,80,481.0,163.0,20.0,200,11.0,12.6,51.0,17.0,Guard,46.51


In [13]:
best_wing = sortwing.drop_duplicates(subset='player', keep='first')
best_wing.head(10)

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
19925,19926,2006,Andrei Kirilenko,SF,UTA,69,391.0,102.0,220.0,162,1.0,12.3,67.0,24.0,Wing,89.66
21908,21909,2009,LeBron James,SF,CLE,81,507.0,137.0,93.0,139,15.0,21.2,55.0,13.0,Wing,69.82
26525,26526,2017,Giannis Antetokounmpo,SF,MIL,80,558.0,131.0,151.0,246,1.0,3.2,103.0,7.0,Wing,67.93
20913,20914,2007,Shawn Marion,SF,PHO,80,613.0,156.0,122.0,216,9.7,8.3,115.0,11.0,Wing,65.98
28110,28111,2019,Paul George,SF,OKC,77,523.0,170.0,34.0,214,7.8,16.5,61.0,30.0,Wing,65.08
20720,20721,2007,Josh Smith,SF,ATL,72,456.0,101.0,207.0,241,-4.0,4.0,128.0,25.0,Wing,64.76
26059,26060,2016,Kawhi Leonard,SF,SAS,72,398.0,128.0,71.0,133,14.3,7.1,50.0,11.0,Wing,60.08
24240,24241,2013,Kevin Durant,SF,OKC,81,594.0,116.0,105.0,143,11.3,7.2,75.0,14.0,Wing,58.99
17182,17183,2000,Tracy McGrady,SF,TOR,79,313.0,90.0,151.0,201,0.4,0.1,81.0,7.0,Wing,56.78
21000,21001,2008,Andre Iguodala,SF,PHI,82,361.0,171.0,49.0,187,-0.2,-1.8,91.0,13.0,Wing,56.16


In [14]:
best_big = sortbig.drop_duplicates(subset='player', keep='first')
best_big.head(10)

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,pos_group,score
16866,16867,2000,Dikembe Mutombo,C,ATL,82,853.0,27.0,269.0,248,-1.5,13.1,103.0,19.0,Big,329.95
21349,21350,2008,Marcus Camby,C,DEN,79,807.0,85.0,285.0,216,3.8,1.0,145.0,33.0,Big,323.25
21156,21157,2008,Dwight Howard,C,ORL,82,882.0,74.0,176.0,274,7.5,8.0,133.0,8.0,Big,311.90
25235,25236,2015,DeAndre Jordan,C,LAC,82,829.0,81.0,183.0,245,11.8,15.4,128.0,4.0,Big,299.10
26936,26937,2018,Andre Drummond,C,DET,78,848.0,114.0,127.0,250,-1.2,-3.1,88.0,8.0,Big,295.50
17141,17142,2000,Shaquille O'Neal,C,LAL,79,742.0,36.0,239.0,255,10.4,10.9,144.0,12.0,Big,284.10
26533,26534,2017,Hassan Whiteside,C,MIA,77,795.0,57.0,161.0,226,1.9,1.9,108.0,12.0,Big,282.30
20741,20742,2007,Kevin Garnett,PF,MIN,76,792.0,89.0,126.0,184,-0.5,14.8,80.0,15.0,Big,277.05
26813,26814,2017,Rudy Gobert,C,UTA,81,721.0,49.0,214.0,246,8.2,11.6,127.0,16.0,Big,273.50
31169,31170,2024,Anthony Davis,C,LAL,76,722.0,91.0,178.0,177,2.3,6.0,117.0,4.0,Big,267.60
